In [421]:
import numpy as np
import pandas as pd
from scipy.signal import butter, lfilter
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [423]:
# Load OpenBCI CSV with no header
df = pd.read_csv('BrainFlow-RAW_JUMP_0.csv', header=None, delimiter = '\t')

# EEG = columns 1–8 (0-indexed)
eeg = df.iloc[:, 1:9].values.astype(float)

accel = df.iloc[:, 9:12].values.astype(float)
timestamps = df.iloc[:, 22].values

# Handle bad offset: remove mean per channel
eeg = eeg - np.mean(eeg, axis=0)

In [425]:
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    return butter(order, [lowcut / nyq, highcut / nyq], btype='band')

def bandpass_filter(data, lowcut=0.5, highcut=40.0, fs=250.0, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    return lfilter(b, a, data, axis=0)

fs = 250  # Hz sample rate
filtered_eeg = bandpass_filter(eeg, fs=fs)

In [427]:
nod_interval_sec = 5
nod_duration_sec = 1
labels_nod = np.zeros(len(eeg))

for start in range(0, len(eeg), nod_interval_sec * fs):
    end = start + nod_duration_sec * fs
    if end <= len(labels_nod):
        labels_nod[start:end] = 1

In [429]:
def extract_features(window):
    feats = {}
    for i in range(window.shape[1]):
        ch = window[:, i]
        feats[f'ch{i}_mean'] = np.mean(ch)
        feats[f'ch{i}_std'] = np.std(ch)
        feats[f'ch{i}_rms'] = np.sqrt(np.mean(ch**2))
    return feats

win_size = fs  # 1 second
stride = int(fs * 0.5)  # 50% overlap

X = []
y = []

for start in range(0, len(filtered_eeg) - win_size, stride):
    end = start + win_size
    window = filtered_eeg[start:end]
    label = "nod" if np.any(labels_nod[start:end]) else "nothing"
    feats = extract_features(window)
    X.append(feats)
    y.append(label)

In [431]:
# Load OpenBCI CSV with no header
df2 = pd.read_csv('BrainFlow-RAW_DOUBLEBLINK_0.csv', header=None, delimiter = '\t')

# EEG = columns 1–8 (0-indexed)
eeg2 = df2.iloc[:, 1:9].values.astype(float)

accel2 = df2.iloc[:, 9:12].values.astype(float)
timestamps2 = df2.iloc[:, 22].values

# Handle bad offset: remove mean per channel
eeg2 = eeg2 - np.mean(eeg2, axis=0)

In [433]:
fs = 250  # Hz sample rate
filtered_eeg2 = bandpass_filter(eeg2, fs=fs)

In [435]:
blink_interval_sec = 5
blink_duration_sec = 1
labels_blink = np.zeros(len(eeg2))

for start in range(0, len(eeg2), blink_interval_sec * fs):
    end = start + blink_duration_sec * fs
    if end <= len(labels_blink):
        labels_blink[start:end] = 1
win_size = fs  # 1 second
stride = int(fs * 0.5)  # 50% overlap

for start in range(0, len(filtered_eeg2) - win_size, stride):
    end = start + win_size
    window = filtered_eeg2[start:end]
    label = "blink" if np.any(labels_blink[start:end]) else "nothing"
    feats = extract_features(window)
    X.append(feats)
    y.append(label)

vec = DictVectorizer()
X = vec.fit_transform(X).toarray()
y = np.array(y)

In [437]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(y_pred)
print(classification_report(y_test, y_pred))

['nod' 'nothing' 'nothing' 'nothing' 'nothing' 'nothing' 'nothing'
 'nothing' 'nothing' 'nothing' 'nothing' 'blink' 'nothing' 'nothing'
 'nothing' 'nothing' 'nothing' 'nothing' 'nothing' 'blink' 'nothing'
 'nothing' 'nothing' 'nothing' 'nothing' 'nothing' 'blink' 'nothing'
 'blink' 'nothing' 'nothing' 'nothing' 'nothing' 'nothing' 'nothing'
 'nothing' 'blink' 'nod' 'nod' 'nothing' 'nod' 'nod' 'nothing' 'blink'
 'nothing' 'nothing' 'nothing' 'nothing' 'nothing' 'blink' 'nothing'
 'nothing' 'nod' 'nothing' 'nothing' 'nothing' 'nothing' 'nothing'
 'nothing' 'nothing' 'blink' 'nothing' 'nothing' 'nothing' 'nothing'
 'nothing' 'nod' 'nod' 'nothing' 'nothing' 'nothing' 'nothing' 'nothing'
 'nothing' 'nothing' 'nothing' 'nothing' 'nothing' 'nothing' 'nothing'
 'nothing' 'nothing' 'nothing' 'nothing' 'nod' 'nothing' 'nothing'
 'nothing' 'blink' 'nothing' 'nothing' 'nothing' 'nothing' 'nothing'
 'blink' 'nothing' 'blink' 'nothing' 'nothing' 'nothing']
              precision    recall  f1-score

In [445]:
dfX = pd.DataFrame(X)
dfY = pd.DataFrame(y)

dfX.to_csv('eeg_dataset_input.csv', index=False)
dfY.to_csv('eeg_labels.csv', index=False)